In [ ]:
import numpy as np
import pathlib
import anndata as ad
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import napari

from tqdm.notebook import tqdm

# from shapely improt Polygon
from shapely.geometry import Polygon, MultiLineString, box, Point, MultiPolygon
from shapely.ops import unary_union, polygonize
import shapely

from  geojson.geometry import GeometryCollection
import geojson

import imageio as iio

# Load brain1+brain3 anndata

In [ ]:
# Load brain1 + brain3 combined adata
adata = ad.read_h5ad('//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/brianl/Brain_1_3_TH_ZI.h5ad')

In [ ]:
adata

## Subset by brain, division

In [ ]:
# use .obs['brain']==1 or ==3 to distinguish between brain1 vs brain3 cells
# if .obs['brain'] doesn't exist, you can instead use the codebook names:
# codebook_brain1 = 'VZG147'
# codebook_brain3 = 'wholebrain031822a'

# list of neuronal divisions so we can just focus on neuons
divisions_neuronal = [
                    '1 Pallium glutamatergic',
                    '2 Subpallium GABAergic',
                    '3 PAL-sAMY-TH-HY-MB-HB neuronal',
                    '4 CBX-MOB-other neuronal'
                   ]

# napari for manual annotation

## Generate z coords for napari

### Plot paired sections of brain1 & brain3

Can we use section ID or target_atlas_plate either as the actual z coords or as a way to generate z coords?

In [ ]:
# Plot the sections by their spatial_cirro coords, label with section #s & target_atlas_plate
plt.figure(figsize=(16,9))
np.random.seed(42) # so sampling via random.choice is replicable

for section, gb in adata.obs.groupby("section"):
    # get spatial_cirro coords
    curr_section_locs = adata[adata.obs.section==section].obsm["spatial_cirro"]
    # sample 10% of cells so this doesn't take forever to plot
    sampling_mask = np.random.choice([False, True], len(curr_section_locs), p=[0.90, 0.10])
    # get target_atlas_plate
    target_atlas_plate = gb.loc[:,["target_atlas_plate"]].values.astype(int)[0][0]
    # plot and label
    plt.scatter(curr_section_locs[sampling_mask][:,0], curr_section_locs[sampling_mask][:,1])
    plt.annotate(section, (np.min(curr_section_locs[sampling_mask][:,0]),np.max(curr_section_locs[sampling_mask][:,1])))
    plt.annotate(target_atlas_plate, (np.min(curr_section_locs[sampling_mask][:,0]),np.min(curr_section_locs[sampling_mask][:,1])))

### Convert target_atlas_plate to z coords

In the paired section plot above, you can see that:
1. the minimum target_atlas_plate value labels the same AP section in both brains
2. target_atlas_plate increments on the same interval (every 200um?) even though brain3 was sampled at a lower axial resolution & some sections are missing in both brains
3. ( minimum(brain1 target_atlas_plate) - minimum(brain3 target_atlas_plate) ) = 14

Thus, we can add a simple offset of +14 to the brain3 target_atlas_plate values to align them to the brain1 values

In [ ]:
# To make things more readable, let's use .obs['target_atlas_plate'] to set our
# z coordinates in napari, BUT these values are offset:
# the minimum target_atlas_plate value labels the same AP section in both brains
# and both sets of values increment on the same interval (200um?) even though
# brain3 was samples less

# thus, offset the brain3 target_atlas_plate values by +14 to match brain1 
offset_atlas_plate = 14
brain3_mask = (adata.obs['brain']==3)

# get all the target_atlas_plate values - use copy() so we don't edit adata
target_atlas_plate_vals = adata.obs['target_atlas_plate'].values.copy()

# only adjust if we haven't already offset brain3
if int(np.min(target_atlas_plate_vals))==49:
    target_atlas_plate_vals[brain3_mask] = target_atlas_plate_vals[brain3_mask] + offset_atlas_plate

# generate range array of all target_atlas_plate values to use as a key
unique_plates = np.unique(target_atlas_plate_vals).astype(int)
target_atlas_plate_arange = np.arange(np.min(unique_plates), 
                                     np.max(unique_plates)+1)

# convert aligned target_atlas_plate values to z-coords using
# target_atlas_plate_arange as the key and it's indices as the new z-coords 
napari_z_coords = [np.where(target_atlas_plate_arange==val)[0][0] 
                   for val in target_atlas_plate_vals
                  ]

# save new z-coords to adata
adata.obs['napari_z_brain1and3'] = napari_z_coords

In [ ]:
# Show the sections in their spatial_cirro coords, label with section #s
# target_atlas_plates should not match, but the new napari_z coords should
plt.figure(figsize=(16,9))
np.random.seed(42) # so sampling via random.choice is replicable

for section, gb in adata.obs.groupby("section"):
    # get spatial_cirro coords
    curr_section_locs = adata[adata.obs.section==section].obsm["spatial_cirro"]
    # sample 10% of cells so this doesn't take forever to plot
    sampling_mask = np.random.choice([False, True], len(curr_section_locs), p=[0.90, 0.10])
    # get target_atlas_plate
    target_atlas_plate = gb.loc[:,["target_atlas_plate"]].values.astype(int)[0][0]
    # get new z-coords
    z_coord = gb.loc[:,["napari_z_brain1and3"]].values.astype(int)[0][0]
    # plot and label
    plt.scatter(curr_section_locs[sampling_mask][:,0], curr_section_locs[sampling_mask][:,1])
    plt.annotate(section, (np.min(curr_section_locs[sampling_mask][:,0]),np.max(curr_section_locs[sampling_mask][:,1])))
    plt.annotate(target_atlas_plate, (np.min(curr_section_locs[sampling_mask][:,0]),np.min(curr_section_locs[sampling_mask][:,1])))
    plt.annotate(z_coord, (np.min(curr_section_locs[sampling_mask][:,0]),np.min(curr_section_locs[sampling_mask][:,1])-1000))

## Convert XY coords for napari & add to adata.obs

In [ ]:
# weird things will happen if this has already been run in this notebook, so
# only generate yx coords if they're not there
if 'napari_y_brain1and3' not in adata.obs.columns:
    # Get cell ZYX coords from 'spatial_rotated'
    xy_locs = adata.obsm["spatial_rotated"]
    
    # reverse x and y to match napari expected YX order
    yx_locs = xy_locs[:,::-1]
    # flip sign so brains display right-side-up
    yx_locs[:,0] = -yx_locs[:,0]
    
    # offset brain1 y-coords (with negative #) so it's on top in napari viewer 
    # (positive number will put it on bottom instead)
    napari_brain1_y_offset = -4000
    yx_locs[adata.obs.brain==1, 0] = yx_locs[adata.obs.brain==1, 0] + napari_brain1_y_offset
    
    # Store adjusted napari zyx coordinates in the anndata object
    adata.obs['napari_y_brain1and3'] = yx_locs[:,0]
    adata.obs['napari_x_brain1and3'] = yx_locs[:,1]

## Annotate MD in a napari viewer

Starting copy-paste from Brian's notebook: [TH_atlas_data_annotation.ipynb](https://github.com/AllenInstitute/merscope_notebooks/blob/thalamus/project_specific/TH/TH_atlas_data_annotation.ipynb)

In [ ]:
# Generate a set of random colors, one for each of the subclasses 
# & clusters we're going to plot
random_colors = np.clip(np.random.rand(1000,4)-0.2, a_max=1.0, a_min=0)
random_colors[:,3]=1.0

color_iter = 0  # counter so we can 

In [ ]:
v = napari.Viewer()
v.theme = 'light' # so we can use these on a white slides

In [ ]:
# Display all neuronal subclasses
for subclass, gb in adata.obs.groupby("subclass_id_label"):
    # Only display neuronal cells
    if gb.division_id_label[0] not in divisions_neuronal:
        continue
    # Only display subclasses with at least 50 cells (I think that's what this is??)
    if gb.shape[0]<50:
        continue

    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1) 

    # Show points in napari Viewer window
    v.add_points(all_locs, name=subclass, edge_width=0, size=10,
                 face_color=random_colors[color_iter,:])
    color_iter +=1

In [ ]:
# Display clusters from "Prong 1 Vitessce links by nucleus" spreadsheet
md_clusters = [cluster for cluster in adata.obs['cluster_label'].unique() if (('1132' in cluster) | ('1133' in cluster))]

for cluster, gb in adata.obs.groupby('cluster_label'):
    # only display the MD clusters
    if (not ('1132' in cluster)) and (not ('1133' in cluster)):
        continue
    
    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1)  

    # Show points in napari Viewer window
    v.add_points(all_locs, name=cluster, edge_width=0, size=10,
                 face_color=random_colors[color_iter,:])
    color_iter +=1

In [ ]:
# Manually add new shapes layer using the Napari GUI interface
# Draw manual annotations using the polygon tool

# # If you want to change the display properties of the polygon, use these features
# shapes_layer_to_edit = v.layers[-1]
# shapes_layer_to_edit.edge_width = 10
# shapes_layer_to_edit.edge_color = 'black'
# shapes_layer_to_edit.face_color = '#ffffff00'
# shapes_layer_to_edit.refresh()

### Take screenshots of annotations

In [ ]:
# # Save screenshots
# write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
# sub_dir = ''
# filename_base = '/MD_annotations_brain_1_3'

# all_z_pos_array = np.unique(adata.obs['napari_z_brain1and3']).astype(int)

# for z in all_z_pos_array:
#     v.dims.set_point(0,z)
#     curr_screenshot = v.screenshot()
#     filepath = write_path+filename_base+'_z'+str(z)+'.png'
#     print(filepath)
#     iio.imwrite(filepath, curr_screenshot, format='png')

### Extract shapes & explore

In [ ]:
# shape_layer = v.layers[-1]

# MD_brain1_left = shape_layer.data[0][:,1:]
# MD_brain1_right = shape_layer.data[1][:,1:]
# MD_latest= shape_layer.data[-1][:,1:]

In [ ]:
# Polygon(MD_latest)

In [ ]:
# shape_layer.data

### Save MD annotation shapes as GeoJSON files

In [ ]:
# MD_shapes = v_reload.layers[-1].data

# # MD_shapes
# MD_shapes[0]

In [ ]:
# napari_z_coords = np.zeros(len(MD_shapes))
# MD_shapes_2D = np.empty(len(MD_shapes), dtype=object)

# for i, shape in enumerate(MD_shapes):
#     napari_z_coords[i] = shape[0,0]
#     MD_shapes_2D[i] = shape[:,1:]

In [ ]:
# # Write MD annotation shapes to a GeoJSON file
# write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
# filename = '/your_polygons.geojson'

# MD_polys = [Polygon(shape[:,:]) for shape in MD_shapes]
# gc = geojson.GeometryCollection()
# # Storing YX in "coordinates" and Z in "napari_z_coords"
# gc.geometries.extend([geojson.Polygon(coordinates = [ [shape[ii,1], shape[ii,2]] 
#                                                      for ii in range(shape.shape[0])],
#                                       napari_z_coords = shape[0,0]
#                                      ) for shape in MD_shapes])

# with open(write_path+filename,'w') as w:
#     geojson.dump(gc,w)

In [ ]:
# # Testing that we've stored the z_coords in a way that works for extraction
# with open(write_path+filename,'r') as r:
#     gc = geojson.load(r)

# # Extract the MD shapes from geojson file
# MD_shapes_loaded = [ np.asarray(poly["coordinates"]) for poly in gc["geometries"] ] # idk if they have to be arrays ..
# MD_shapes_loaded
# z_loaded = [ poly["napari_z_coords"] for poly in gc["geometries"] ] # idk if they have to be arrays ..
# z_loaded

# Load existing annotations for napari MD figures

In [ ]:
# Picked colormaps with large perceptual differences between adjacent colors
# to highlight the handful of subclasses & clusters plotting
cmap_0 = plt.get_cmap('Dark2')
cmap_1 = plt.get_cmap('tab10')
rgb_cm = np.concatenate((np.asarray(cmap_0.colors), np.asarray(cmap_1.colors)))
display_colors = np.zeros((rgb_cm.shape[0],4))
display_colors[:,:3] = rgb_cm
display_colors[:,3] = 1
# display_colors

## Plot only relevant subclasses/supertypes/clusters

In [ ]:
v_reload = napari.Viewer()
# napari defaults to a dark theme, which is not ideal for making 
# figures that will go in slides, etc
v_reload.theme = 'light'

In [ ]:
color_iter = 0

# Plot neuronal divisions in grey to show shape of thalamus
divisions_to_plot = ['1 Pallium glutamatergic',
                    '2 Subpallium GABAergic',
                    '3 PAL-sAMY-TH-HY-MB-HB neuronal',
                    '4 CBX-MOB-other neuronal']
groupby_col_name = 'division_id_label'
for division, gb in adata.obs.groupby(groupby_col_name):
    # Only display neuronal cells
    if division not in divisions_to_plot:
        continue

    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1) 

    # Show points in napari Viewer window
    v_reload.add_points(all_locs, name=division, edge_width=0, size=10,
                        face_color=[128/255,128/255,128/255,0.2]) # grey


    
# Which subclasses are helpful in distinguishing MD boundaries?
subclasses_to_plot = ['067 PVT-PT Ntrk1 Glut', # PVT (above MD)
                      '068 CM-IAD-CL-PCN Glut'  # below MD
                     ]
# Plot subclasses
groupby_col_name = 'subclass_id_label'
for subclass, gb in adata.obs.groupby(groupby_col_name):
    # Only display neuronal cells
    if gb.division_id_label[0] not in divisions_neuronal:
        continue
    # Only display subclasses with at least 50 cells (I think that's what this is??)
    if gb.shape[0]<50:
        continue

    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1) 

    # Show points in napari Viewer window
    # color the subclasses I used to make the MD annotations
    if subclass in subclasses_to_plot:
        v_reload.add_points(all_locs, name=subclass, edge_width=0, size=14,
                            face_color=display_colors[color_iter,:])
        color_iter +=1


        
# Which supertypes are helpful?
supertypes_to_plot = ['0290 PF Fzd5 Glut_1', '0291 PF Fzd5 Glut_2', #PF (posterior end of MD)
                      '0273 TH Prkcd Grin2c Glut_12',  # midline
                      '0274 TH Prkcd Grin2c Glut_13',  # lateral/sides of MD
                      '0279 TH Prkcd Grin2c Glut_5',  # lateral/sides of MD
                      '0271 TH Prkcd Grin2c Glut_10',  # inside MD
                      ]
# Plot supertypes
groupby_col_name = 'supertype_id_label'
for supertype, gb in adata.obs.groupby(groupby_col_name):
    # only display the relevant supertypes
    if supertype not in supertypes_to_plot:
        continue

    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1) 

    # plot points
    v_reload.add_points(all_locs, name=supertype, edge_width=0, size=14,
                        face_color=display_colors[color_iter,:])
    color_iter +=1


    
# Which clusters are helpful?
clusters_to_plot = ['1130 TH Prkcd Grin2c Glut_1'  # inside MD and also far away
                   ]
# these clusters belong to supertype '0271 TH Prkcd Grin2c Glut_10', which is
# already plotted above:
# '1132 TH Prkcd Grin2c Glut_10',
# '1133 TH Prkcd Grin2c Glut_10'

# Plot clusters
groupby_col_name = 'cluster_label'
for cluster, gb in adata.obs.groupby(groupby_col_name):
    # only display the MD clusters
    if cluster not in clusters_to_plot:
        continue
    
    # Get napari ZYX coords for displaying brain1 & brain3 together
    all_locs = np.concatenate([gb.loc[:,['napari_z_brain1and3']].values,
                               gb.loc[:,['napari_y_brain1and3']].values, 
                               gb.loc[:,['napari_x_brain1and3']].values], 
                              axis=1) 

    # Show points in napari Viewer window
    v_reload.add_points(all_locs, name=cluster, edge_width=0, size=14,
                 face_color=display_colors[color_iter,:])
    color_iter +=1

In [ ]:
# Load in the MD annotation polygons from the geojson file
# NB: coordinates stored as [z,y,x] to match napari's expected ordering
write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
filename = '/MD_polygons_withZcoords_v230612.geojson'
with open(write_path+filename,'r') as r:
    gc = geojson.load(r)

# merge the YX and Z coordinates into ZYX vertice arrays that napari recognizes
MD_shapes_reload = [ np.concatenate((np.full((len(poly["coordinates"]),1), poly["napari_z_coords"]), 
                                     np.asarray(poly["coordinates"])), 
                                    axis=1)
             for poly in gc["geometries"] ]

In [ ]:
# Add MD annotations to a new Shapes layer in napari
shapes_layer = v_reload.add_shapes()
shapes_layer.add_polygons(
  MD_shapes_reload,
  edge_width=20,
  edge_color=[0,0,0,1],
  face_color=[1,1,1,0]
)

## Save new geojson, screenshots IFF manually updated annotations

In [ ]:
# # If you manually adjusted the annotations in napari, save to new geojson file

# # Get 3D polygons from napari Shapes layer
# MD_shapes_3D_adjusted = v_reload.layers[-1].data  # ZYX order

# # # if you need to translate the polygons in Y, use this code chunk:
# # MD_shapes_to_update = MD_shapes_3D_adjusted.copy()
# # for i, shape in enumerate(MD_shapes_to_update):
# #     if MD_shapes_to_update[i][0,1] > 3000:
# #         MD_shapes_to_update[i][:,1] = MD_shapes_to_update[i][:,1]-8000

# # Write MD annotation shapes to a GeoJSON file
# write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
# filename = '/MD_polygons_withZcoords_v230613.geojson'

# # Storing YX in "coordinates" and Z in "napari_z_coords"
# gc = geojson.GeometryCollection()
# gc.geometries.extend([geojson.Polygon(coordinates = [ [shape[ii,1], shape[ii,2]] 
#                                                      for ii in range(shape.shape[0])],
#                                       napari_z_coords = shape[0,0]
#                                      ) for shape in MD_shapes_3D_adjusted])

# with open(write_path+filename,'w') as w:
#     geojson.dump(gc,w)

In [ ]:
# # Save screenshots of the updated annotations
# write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
# sub_dir = '/light_theme_latest'
# filename_base = '/MD_annotations_brain_1_3'

# all_z_pos_array = np.unique(adata.obs['napari_z_brain1and3']).astype(int)

# for z in all_z_pos_array:
#     v_reload.dims.set_point(0,z)
#     curr_screenshot = v_reload.screenshot()
#     filepath = write_path+sub_dir+filename_base+'_z'+str(z)+'.png'
#     # print(filepath)
#     iio.imwrite(filepath, curr_screenshot, format='png')

# Add manual annotation polygons to anndata.uns

In [ ]:
# Add MD annotations to anndata
# Load in the MD annotation polygons from the geojson file
write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
filename = '/MD_polygons_withZcoords_v230612.geojson'
with open(write_path+filename,'r') as r:
    gc = geojson.load(r)

# convert to a string
MD_shapes_str = geojson.dumps(gc)

# add that string to the unstructured portion of the anndata
adata.uns['MD_polygons'] = MD_shapes_str

# # to reload the string, use:
# gc_reloaded = geojson.loads(adata.uns['MD_annotation'])

# Load MD annotations from geojson file

In [ ]:
# Load geojson file
write_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/MD_annotations'
filename = '/MD_polygons_withZcoords_v230612.geojson'
with open(write_path+filename,'r') as r:
    gc = geojson.load(r)

In [ ]:
# Get the MD annotation info we need from the geojson file
MD_polys = [shapely.geometry.Polygon(poly["coordinates"]) for poly in gc["geometries"]]  #YZ coords stored as shapely polygons (expected by cells_in_polygon())
MD_shapes_z = [ poly["napari_z_coords"] for poly in gc["geometries"] ]  # Z coords

# Determine which cells are in MD polygons

## cells_in_polygon()

In [ ]:
# method to find cells that are within the bounds of a polygon
# (borrowed from Brian)
def cells_in_polygon(cell_locations, polygon,
                     checkBoundingBox = False):
    """
    checks numpy array of point locations and determines if they're inside the  
    polygon
    - currently only checks in 2D
    - uses shapely's contains() method

    Args:
        cell_locations is a numpy array size n_cells X 2  
        polygon is a shapely.geometry.Polygon, with YX coordinates
        checkBoundingBox if True, use bounding box coordinates to avoid making 
                         a containsPoint call.

    Returns:
        1D numpy array of Booleans indicating if the cells are in the polygon
    """
    
    # constrain the search area to just the polygon's bounding box so it can
    # search for fewer cells and thus save time
    if checkBoundingBox:

        polySpots =[]
        bb_bounds = polygon.bounds
        conditionA = np.logical_or(cell_locations[:,0]< bb_bounds[0] ,
                         cell_locations[:,0]> bb_bounds[2])
        conditionB = np.logical_or(cell_locations[:,1]< bb_bounds[1] ,
                         cell_locations[:,1]> bb_bounds[3]) 

        conditions = np.logical_not( np.logical_or(conditionA, conditionB))
        # this is the subset of spots within the xy bounding box of the polygon:

        okSpots = np.where(conditions)[0]
    else:
        
        okSpots = range(cell_locations.shape[0])

                         
    spotList = []

    for spot_row in okSpots:
        spotList.append( Point(cell_locations[spot_row,0:2]))
            
    is_in_roi = [polygon.contains(spot) for spot in spotList]
                                
    output = np.zeros(cell_locations.shape[0],dtype=bool)
    output[okSpots] = is_in_roi
                            
                                        
    return output

## Assign L vs R to each MD polygon

In [ ]:
# Set up useful array for iterations
ap_locs = sorted(list(adata.obs['napari_z_brain1and3'].unique()))

In [ ]:
# Determine left vs right MD polygons in each section

# get the centroids of each polygon
MD_centers = np.array([p.centroid.coords[0] for p in MD_polys])

# initialize arrays
MD_polys_L_R = np.asarray([None for p in range(len(MD_polys))])
MD_polys_brain_id = np.asarray([None for p in range(len(MD_polys))])

for i, ap_loc in enumerate(ap_locs):
    # keep track of which polygons we're dealing with in this section
    curr_polys_ind = np.where(MD_shapes_z==ap_loc)[0]

    # skip if we have no annotations in this section
    if len(curr_polys_ind)==0:
        continue

    centroids_x = np.asarray([MD_centers[p][1] for p in curr_polys_ind])  # the 1th coord is left-right (X) in napari coords
    centroids_y = np.asarray([MD_centers[p][0] for p in curr_polys_ind])  # the 0th coord is up-down (Y) in napari coords
    
    # Handle the potential presence of both brain1 & brain3 MD annotations
    # if we just have one brain's MD annotated
    if len(curr_polys_ind)==2:
        # set the brain ID
        # use the brain1 offset to determine which polygons belong to which brain
        # 2023-06-01 MT: brain1 offset = -4000; brain1 polygon centroid Y-coords 
        #   are all in the [-3200,-3900] range; brain3 are in [-320, 301]
        #   So, adding 1000 to the offset to catch all the brain1 polygons
        if centroids_y[0] < (napari_brain1_y_offset+1000):
            MD_polys_brain_id = np.asarray([1 if i in curr_polys_ind 
                                                     else val 
                                            for i, val in enumerate(MD_polys_brain_id)])
        else:
            MD_polys_brain_id = np.asarray([3 if i in curr_polys_ind 
                                                     else val 
                                            for i, val in enumerate(MD_polys_brain_id)])
        
        # set the left-right identity
        right_poly_index = curr_polys_ind[np.argmax(centroids_x)] # larger X is right polygon
        left_poly_index = curr_polys_ind[np.argmin(centroids_x)]  # smaller X is left polygon
        MD_polys_L_R[right_poly_index] = 'right'
        MD_polys_L_R[left_poly_index] = 'left' 
    
    # if we have both brains' MDs annotated
    elif len(curr_polys_ind)==4:
        right_poly_ind = [np.nan, np.nan]
        left_poly_ind = [np.nan, np.nan]
        
        # use the brain1 offset to determine which polygons belong to which brain
        # (see if statement above for detailed explanation)
        brain1_poly_filt = centroids_y < napari_brain1_y_offset+1000 # the 0th coord is up-down in napari coords
        brain1_polys_ind = curr_polys_ind[brain1_poly_filt]
        brain3_polys_ind = curr_polys_ind[~brain1_poly_filt]

        # set brain IDs
        MD_polys_brain_id = np.asarray([1 if i in brain1_polys_ind else val for i, val in enumerate(MD_polys_brain_id)])
        MD_polys_brain_id = np.asarray([3 if i in brain3_polys_ind else val for i, val in enumerate(MD_polys_brain_id)])
        
        # determine left-right for ...
        # brain1
        brain1_centroids_x = centroids_x[brain1_poly_filt]
        right_poly_ind[0] = brain1_polys_ind[np.argmax(brain1_centroids_x)] # larger X is right polygon
        left_poly_ind[0] = brain1_polys_ind[np.argmin(brain1_centroids_x)]  # smaller X is left polygon
        # brain3
        brain3_centroids_x = centroids_x[~brain1_poly_filt]
        right_poly_ind[1] = brain3_polys_ind[np.argmax(brain3_centroids_x)] # larger X is right polygon
        left_poly_ind[1] = brain3_polys_ind[np.argmin(brain3_centroids_x)]  # smaller X is left polygon

        # set left-right identity
        MD_polys_L_R = np.asarray(['right' if i in right_poly_ind else val for i, val in enumerate(MD_polys_L_R)])
        MD_polys_L_R = np.asarray(['left' if i in left_poly_ind else val for i, val in enumerate(MD_polys_L_R)])
        
    else:
        print('Warning: Too many or too few MD annotations')  

# print('MD_polys_brain_id:', MD_polys_brain_id)
# print('MD_polys_L_R:', MD_polys_L_R)

In [ ]:
# Sanity checking that we have:
# (a) right number of polys from each brain (brain1=9, brain3=4)
# (b) same number of left and right polys per brain
brain1_polys_LR = MD_polys_L_R[MD_polys_brain_id==1]
brain3_polys_LR = MD_polys_L_R[MD_polys_brain_id==3]

print('brain1 MD right polys:', list(brain1_polys_LR).count('right'))
print('brain1 MD left polys:', list(brain1_polys_LR).count('left'))
print('brain3 MD right polys:', list(brain3_polys_LR).count('right'))
print('brain3 MD left polys:', list(brain3_polys_LR).count('left'))

## Assign cells to poly, give L-R label

In [ ]:
# initialize new .obs columns that we're going to fill
adata.obs['is_in_MD'] = False
adata.obs['left_right_MD'] = 'not in MD'

# For each AP section, assign cells to the MD and then the left or right polygon
for i, ap_loc in enumerate(ap_locs):

    # Get the ADJUSTED napari coordinates for each cell in this section so the
    # cells and polygons are in the same coordinates
    cell_locs = adata.obs.loc[adata.obs.napari_z_brain1and3==ap_loc, ['napari_y_brain1and3', 'napari_x_brain1and3']].values

    # For each MD polygon, find all the cells that fall inside
    for j, poly in enumerate(MD_polys):

        # only procede to checking if cells are in polygon, IFF current poly is
        # in the same ap section as current cell_locs
        ap = MD_shapes_z[j]
        if ap != ap_loc:
            continue
            
        cell_is_in_MD = cells_in_polygon(cell_locs, poly, checkBoundingBox=True)

        # Update 'is_in_MD' to flag cells found to be in this MD polygon 
        # BUT don't overwrite if already confirmed to be in a previous MD polygon
        adata.obs.loc[adata.obs['napari_z_brain1and3']==ap_loc,'is_in_MD'] = np.logical_or(adata.obs.loc[adata.obs['napari_z_brain1and3']==ap_loc,"is_in_MD"], cell_is_in_MD)

        # Update 'left_right_MD' field for cells in the MD in this section
        multimask = (adata.obs.napari_z_brain1and3==ap_loc).values
        multimask[multimask.nonzero()] = cell_is_in_MD
        adata.obs.loc[np.logical_and(adata.obs.napari_z_brain1and3==ap_loc,multimask),'left_right_MD'] = MD_polys_L_R[j]
        

### Sanity checking left-right assignment

In [ ]:
# sanity checking all this left-right stuff....
plt.figure(figsize=(12,9))
for ii in range(len(MD_polys_L_R)):
    # print('polygon # ',ii)
    if MD_polys_L_R[ii] =='right':
        mask = np.logical_and(adata.obs.napari_z_brain1and3.astype(int) == int(MD_shapes_z[ii]), 
                               adata.obs.left_right_MD=="right")
        # print('mask sum:', sum(mask))
        xy = adata.obs.loc[mask, ["napari_y_brain1and3","napari_x_brain1and3"]].values
        # change signs on both x & y coords to convert from napari to plt expectations 
        plt.plot(xy[::10,1], -xy[::10,0],'.r')
        plt.plot(MD_centers[ii][1], -MD_centers[ii][0], "*k")
        plt.plot(np.array(MD_polys[ii].exterior.coords)[:,1], -np.array(MD_polys[ii].exterior.coords)[:,0], 'k')

    else:
        mask = np.logical_and(adata.obs.napari_z_brain1and3.astype(int) == int(MD_shapes_z[ii]),
                              adata.obs.left_right_MD=="left")
        xy = adata.obs.loc[mask, ["napari_y_brain1and3","napari_x_brain1and3"]].values
        # change signs on both x & y coords to convert from napari to plt expectations
        plt.plot(xy[::10,1], -xy[::10,0],'.g')
        plt.plot(MD_centers[ii][1], -MD_centers[ii][0], "*k")
        plt.plot(np.array(MD_polys[ii].exterior.coords)[:,1], -np.array(MD_polys[ii].exterior.coords)[:,0], 'k')

plt.axis('equal');

### Save per-polygon L-R info to adata.uns

In [ ]:
# These will come in handy when we want to plot things from just the adata file
# (e.g. inside a CodeOcean capsule)

adata.uns['MD_polys_left_right'] = MD_polys_L_R
adata.uns['MD_polys_brain'] = MD_polys_brain_id

# Collapse all MD polygons together

We want to generate an "average MD" (across animals, sections, and hemispheres) so that we can ask questions about the average spatial patterns of cell types and gene expression that we hope is more informative than just looking at noisy individual sections 

## Orient left & right MDs the same

In [ ]:
# set up variables needed to plot the MD regions

# random colors that will show up well on a white background (-0.2 to darken)
random_colors = np.clip(np.random.rand(1000,4)-0.2, a_max=1.0, a_min=0)
random_colors[:,3]=1.0

# Find all the clusters that are represented in the MD
MD_clusters = adata.obs.loc[adata.obs.is_in_MD,"cluster_label"].unique()
MD_clusters = [cl for cl in MD_clusters if adata.obs.loc[adata.obs.cluster_label==cl,"division_id_label"].values[0] in divisions_neuronal]
MD_clusters = np.asarray(sorted(list(MD_clusters)))
# MD_clusters

In [ ]:
plt.figure(figsize=(20,9))
shift_for_plot = 1000  # to line all the MDs up in a row to see them at once
MD_values_list = []

# Flip each "left" polygon along the midline to match the "right" polygons, then
# display each polygon so we can see that we've done this re-orienting correctly
for i in range(len(MD_polys_L_R)):
    # get the correct cells from this polygon
    # first check for left vs right
    if MD_polys_L_R[i] =='right':
        zLR_mask = np.logical_and(adata.obs.napari_z_brain1and3==MD_shapes_z[i], 
                                  adata.obs.left_right_MD=="right")
    else:
        zLR_mask = np.logical_and(adata.obs.napari_z_brain1and3==MD_shapes_z[i],
                                  adata.obs.left_right_MD=="left")
    # then check for brain1 vs brain3
    if MD_polys_brain_id[i] ==1:
        zLRbrain_mask = np.logical_and(zLR_mask, adata.obs.brain==1)
    else:
        zLRbrain_mask = np.logical_and(zLR_mask, adata.obs.brain==3)
        
    xy = adata.obs.loc[zLRbrain_mask, ["napari_y_brain1and3","napari_x_brain1and3"]].values
    

    # Find distance of each selected cell from center of the current MD polygon
    center = MD_centers[i]
    xy_from_center = np.zeros([xy.shape[0], 4])
    xy_from_center[:,0] = xy[:,0]-center[0]
    
    # flop left side over to right
    if MD_polys_L_R[i] =='right':
        xy_from_center[:,1] = xy[:,1]-center[1]
    else:
        xy_from_center[:,1] = -(xy[:,1]-center[1])
    
    # calculate polar coordinates
    xy_from_center[:,2] = np.sqrt(xy_from_center[:,0]**2 + xy_from_center[:,1]**2 )
    xy_from_center[:,3] = np.arctan2(xy_from_center[:,1],xy_from_center[:,0])
    
    # save YX and polar coordinates to anndata
    adata.obs.loc[zLRbrain_mask,["napari_y_from_center","napari_x_from_center","r_from_center","angle_from_center"]] = xy_from_center

    # plot clusters in MD to make sure we've calculated the coordinates correctly
    for cl in adata.obs.loc[zLRbrain_mask,:].cluster_label.unique():
        if "NN" in cl:
            continue
        if not any([(a in cl) for a in ["1132","1133","1130"]]):
            clmask = adata.obs.loc[zLRbrain_mask,["cluster_label"]]==cl

            plt.plot(xy_from_center[clmask.values.flatten(),1]+(shift_for_plot*i), 
                     xy_from_center[clmask.values.flatten(),0],
                     '.', color = 'gray', markersize=1)
        else:
            color_index = np.where(np.array(MD_clusters)==cl)[0][0]

            clmask = adata.obs.loc[zLRbrain_mask,["cluster_label"]]==cl
            plt.plot(xy_from_center[clmask.values.flatten(),1]+(shift_for_plot*i), 
                     xy_from_center[clmask.values.flatten(),0],
                     '.', color = random_colors[color_index,:], markersize=1)
plt.axis([-500, 25500, -1000, 1000])
plt.gca().set_aspect('equal', adjustable='box')

# Save new h5ad with MD polygon info

We need to store 7 key categories of metadata in the anndata object to be able to use them later in other notebooks (e.g. codeocean):

1. per cell (stored as columns in adata.obs):
    1. 'napari_z_brain1and3','napari_y_brain1and3','napari_x_brain1and3': ZYX coordinates adjusted for displaying brain1 & brain3 in napari
    2. 'is_in_MD': boolean for whether or not the cell in is an MD polygon
    3. 'left_right_MD': indicates whether each cell is in the 'left' or 'right' MD polygon or 'not_in_MD' at all
    4. 'napari_y_from_center','napari_x_from_center','r_from_center','angle_from_center: cartesian & polar coordinates, using the center of the MD polygon as a reference
2. per MD annotation polygon (stored in adata.uns):
    1. 'MD_polygons': Shapely Polygons of the manually annotated MD regions, stored as a string of a geojson object
    2. 'MD_polys_left_right': L/R MD assignment for each polygon
    3. 'MD_polys_brain': brain assignment for each polygon

In [ ]:
adata.obs.columns

In [ ]:
filename = 'your_updated_anndata_with_annotations.h5ad'
adata.write('//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/thalamus_data/'+filename, compression="gzip")